In [1]:
# Initial setup by importing needed modules
import pandas as pd
import numpy as np
import datetime as dt
pd.options.display.max_columns = None
pd.options.display.max_rows=None

In [2]:
# Read off data and put that into a panda dataframe
lend = pd.read_csv('data/LoanStats3a.csv', header=1, low_memory=False)

#### Understanding data structure, checking shape and data type

In [3]:
print(lend.shape)
lend.info()

(42538, 144)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42538 entries, 0 to 42537
Columns: 144 entries, id to settlement_term
dtypes: float64(115), object(29)
memory usage: 46.7+ MB


There are 144 columns, 115 of those are floats. Let's look at floats columns first. 

In [4]:
lend.select_dtypes(include='float').head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,url,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
0,NaN,5000.0,5000.0,4975.0,162.87,24000.0,NaN,27.65,0.0,1.0,NaN,NaN,3.0,0.0,13648.0,9.0,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,171.62,0.0,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2500.0,2500.0,2500.0,59.83,30000.0,NaN,1.00,0.0,5.0,NaN,NaN,3.0,0.0,1687.0,4.0,0.0,0.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,119.66,0.0,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2400.0,2400.0,2400.0,84.33,12252.0,NaN,8.72,0.0,2.0,NaN,NaN,2.0,0.0,2956.0,10.0,0.0,0.0,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.00,649.91,0.0,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,10000.0,10000.0,10000.0,339.31,49200.0,NaN,20.00,0.0,1.0,35.0,NaN,10.0,0.0,5598.0,37.0,0.0,0.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.00,357.48,0.0,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,3000.0,3000.0,3000.0,67.79,80000.0,N

While they are relevant information on describing the loans. We observed two issues:
1. The problem is to predict the loan default/paid off chances at the point of application. Details about the loans (such as terms, interest rates, hardship_length, etc.) are unknown upfront. Those columns cannot be used in the machine learning analysis.
2. Some columns appears to be all missing values. This bring us concerns on whether the dataset is complete. We will look into that soon.

Now we will review the 29 object columns. 

In [5]:
lend.select_dtypes(exclude='float').head()

,id,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type,hardship_flag,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date
0,NaN,36 months,10.65%,B,B2,NaN,10+ years,RENT,Verified,Dec-2011,Fully Paid,n,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,Jan-1985,83.7%,f,Jan-2015,NaN,Dec-2018,Individual,N,N,NaN,NaN,NaN
1,NaN,60 months,15.27%,C,C4,Ryder,< 1 year,RENT,Source Verified,Dec-2011,Charged Off,n,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,Apr-1999,9.4%,f,Apr-2013,NaN,Oct-2016,Individual,N,N,NaN,NaN,NaN
2,NaN,36 months,15.96%,C,C5,NaN,10+ years,RENT,Not Verified,Dec-2011,Fully Paid,n,NaN,small_business,real estate business,606xx,IL,Nov-2001,98.5%,f,Jun-2014,NaN,Jun-2017,Individual,N,N,NaN,NaN,NaN
3,NaN,36 months,13.49%,C,C1,AIR RESOURCES BOARD,10+ years,RENT,Source Verified,Dec-2011,Fully Paid,n,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,Feb-1996,21%,f,Jan-2015,NaN,Apr-2016,Individual,N,N,NaN,NaN,NaN
4,NaN,60 months,12.69%,B,B5,University Medical Group,1 year,RENT,Source Verified,Dec-2011,Fully Paid,n,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,Jan-1996,53.9%,f,Jan-2017,NaN,Apr-2018,Individual,N,N,NaN,NaN,NaN


Some of the columns actually have date time or percentage values, but panda cannot recognize them in the initial load. Those columns will need data transformation.

#### Handling missing values

In [6]:
# Look at how many missing records by columns
lend.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,url,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,0.0,42535.000000,42535.000000,42535.000000,42535.000000,4.253100e+04,0.0,42535.000000,42506.000000,42506.000000,15609.000000,3651.000000,42506.000000,42506.000000,4.253500e+04,42506.000000,42535.0,42535.0,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42390.0,0.0,42535.0,0.0,0.0,0.0,42506.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42390.0,42506.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41170.000000,42430.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.000000,160.000000,160.000000
mean,NaN,11089.722581,10821.585753,10139.938785,322.623063,6.913656e+04,NaN,13.373043,0.152449,1.081424,35.017618,59.175568,9.343951,0.058156,1.429786e+04,22.124406,0.0,0.0,12020.388061,11312.927179,9675.675936,2239.981444,1.516890,103.213884,14.423558,2613.245652,0.0,NaN,1.0,NaN,NaN,NaN,0.000094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.143039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.045227,0.000024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4270.794250,49.889875,1.262500
std,NaN,7410.938391,7146.914675,7131.598014,208.927216,6.409635e+04,NaN,6.726315,0.512406,1.527455,22.418427,47.145396,4.496274,0.245713,2.201844e+04,11.592811,0.0,0.0,9094.855348,9038.619934,7105.750304,2585.057393,7.830064,736.904244,162.335958,4385.066535,0.0,NaN,0.0,NaN,NaN,NaN,0.009700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,29.359579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.208737,0.004855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3119.543917,15.569143,4.142869
min,NaN,500.000000,500.00

The dataset has a total of 42K row of data. Some columns almost have no data at all (count = 0), and wouldn't be useful. By scanning the missing records range, I will make the decision to drop columns that have >40K missing records (in other word, records counts <2k). Loan_amnt has three missing records (which are blank rows), we will drop those as well.

In [7]:
lend_drop = lend.dropna(axis=1, how='all', thresh=40000).dropna(subset=['loan_amnt'])
lend_drop.shape

(42535, 51)

#### Why some columns have almost no data?

By looking at the columns, I come up with two reasons:
1. Sensitive information - columns such as member id are personal privacy. Lending club removed those information on purpose.
2. Information not applicable on closed loans. Column such as settlement amount may only be applicable to active loans which are still in progress on being repaid and not for loans that have been paid off (or default).

Fortunately, the two type of columns above do not appear to be relevant to the machine learning problems as they cannot be known in advance or generic information (e.g. members id) that do not have predicting power.

For the float type columns, let's have another look into what those columns are

In [8]:
lend_drop.select_dtypes(include='float').isnull().sum()

loan_amnt                        0
funded_amnt                      0
funded_amnt_inv                  0
installment                      0
annual_inc                       4
dti                              0
delinq_2yrs                     29
inq_last_6mths                  29
open_acc                        29
pub_rec                         29
revol_bal                        0
total_acc                       29
out_prncp                        0
out_prncp_inv                    0
total_pymnt                      0
total_pymnt_inv                  0
total_rec_prncp                  0
total_rec_int                    0
total_rec_late_fee               0
recoveries                       0
collection_recovery_fee          0
last_pymnt_amnt                  0
collections_12_mths_ex_med     145
policy_code                      0
acc_now_delinq                  29
chargeoff_within_12_mths       145
delinq_amnt                     29
pub_rec_bankruptcies          1365
tax_liens           

After reviewing the data dictionary for those columns, I conclude that the missing values would imply zeros. Apply fillna(0) mention to them during the data transformation steps.

#### Data Transformation

For the object columns, further data type transformations are needed. We will focus on issue_d, emp_length, earliest_cr_line, revol_util and title. Other columns are either unknown variables for machine learning or no data type conversions are needed.

#### 1. Loan issues date - convert into datetime value

In [10]:
#use assign function to avoid security type warining
lend_drop = lend_drop.assign(loan_start_d=pd.to_datetime(lend_drop.issue_d, format='%b-%Y'))

#### 2. Employment duration
- convert from text to numeric value. 
- clean up na value, assume <1 year if info is not provided (currently unemployment)

In [11]:
lend_drop['yr_emp'] = lend_drop['emp_length'].str[0:2].replace('< ',0).astype('float')

#### 3. Credit History
- Convert earliest_cr_line into from text to datetime values
- Replace missing values with zeros
- Extract year in order to obtain categorical values

In [12]:
lend_drop.earliest_cr_line = pd.to_datetime(lend_drop.earliest_cr_line, format='%b-%Y')
lend_drop['yr_credit']= (2011-lend_drop.earliest_cr_line.dt.year).fillna(0)

#### 4. Credit line % usage
- Convert to float
- Replace missing values with zeros

In [13]:
lend_drop['revol_util_dec'] = lend_drop['revol_util'].str.replace(r'%',r'0').astype('float')/100

#### 5. Title

In [14]:
# Preview on information in this column
lend_drop.title.head(15)

0                                  Computer
1                                      bike
2                      real estate business
3                                  personel
4                                  Personal
5     My wedding loan I promise to pay back
6                                      Loan
7                           Car Downpayment
8      Expand Business & Buy Debt Portfolio
9               Building my credit history.
10               High intrest Consolidation
11                            Consolidation
12                                  freedom
13                            citicard fund
14                               Other Loan
Name: title, dtype: object

Title describes the purpose of the loan and is useful information. However, in the same dataset, there is another column called 'purpose' which resembles information under Title. The purpose column has better precise label. Hence, we won't be using Title column.

In [15]:
lend_drop.purpose.value_counts()

debt_consolidation    19776
credit_card            5477
other                  4425
home_improvement       3199
major_purchase         2311
small_business         1992
car                    1615
wedding                1004
medical                 753
moving                  629
house                   426
educational             422
vacation                400
renewable_energy        106
Name: purpose, dtype: int64

#### Obtain Target variable

loan_status will serve as target for this analysis. Find out what the type of phrases exist in the loan_status column

In [16]:
lend_drop.loan_status.value_counts()

Fully Paid                                             34116
Charged Off                                             5670
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Name: loan_status, dtype: int64

Per research through discussion forum, "Dose not meet the credit policy' are loans that had issued before Lending Club update credit policy. For the purpose of this study, we will treat them as "Fully Paid" or "Charged Off', regardless of credit policy compliance.

Convert text value feature into numerics for model training purposes. Create a dictionary to convert loan status into 0 (default) or 1 (paid off)

In [17]:
loan_dict = {'Fully Paid': 1, 'Charged Off': 0, 'Does not meet the credit policy. Status:Fully Paid': 1, 'Does not meet the credit policy. Status:Charged Off': 0 }

Create target column by mapping loan status to dictionary,

In [18]:
#use assign function to avoid security type warining
lend_drop = lend_drop.assign(target=lend_drop.loan_status.map(loan_dict))

Save cleaned data set into a csv file for quicker access.

In [19]:
lend_drop.to_csv('data/lending_clean.csv', index=False)

[Back to README](https://github.com/sittingman/lending_repayment)